In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import os, sys
sys.path.insert(0,'../python/')
from analysis_utils import *

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set(font_scale = 2)
sns.set_style("whitegrid")

In [ ]:
import sys
from datawand.parametrization import ParamHelper
ph = ParamHelper("../","LNGraph",sys.argv)

In [ ]:
experiment_id = ph.get("sim_res_dir")
snapshots = ph.get("snapshots")
simulation_dir = ph.get("sim_root_dir")

# 1. Load data

In [ ]:
node_names = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/node_names.csv")

In [ ]:
node_names.head()

In [ ]:
LNBIG_nodes = list(node_names[node_names["is_lnbig"]]["pub_key"])
len(LNBIG_nodes)

In [ ]:
node_names = node_names[["name","pub_key"]]

#experiment_id = ph.get("sim_dir")
experiment_id = "2019-09-06_22:03:19_50000sat_k6000"
snapshots = range(40)#range(54)
simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_1days/"

In [ ]:
experiment_folders = get_experiment_files(experiment_id, snapshots, simulation_dir)

pricing_pol = load_data(experiment_folders, snapshots, "opt_fees")

In [ ]:
source_fee = load_data(experiment_folders, snapshots, "source_fees")

In [ ]:
router_income = load_data(experiment_folders, snapshots, "router_incomes")

## Router incomes

In [ ]:
router_income_col = "fee"#"income"

In [ ]:
x = snapshots
y = [router_income[i][router_income_col].mean() for i in snapshots]
plt.plot(x,y,"bo")

In [ ]:
sns.jointplot(data=pd.concat(router_income), x="snapshot_id", y=router_income_col)

In [ ]:
all_router_incomes = pd.concat(router_income)

## Source fees

In [ ]:
x = snapshots
y = [source_fee[i]["mean_fee"].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
sns.jointplot(data=pd.concat(source_fee), x="snapshot_id", y="mean_fee")

## Optimal pricing

x = snapshots
y = [pricing_pol[i]["opt_delta"].mean() for i in snapshots]
plt.plot(x,y, "bo")

x = snapshots
y = [pricing_pol[i]["opt_traffic"].mean() for i in snapshots]
plt.plot(x,y, "bo")

x = snapshots
y = [pricing_pol[i]["origi_income"].mean() for i in snapshots]
plt.plot(x,y, "bo")

x = snapshots
y = [pricing_pol[i]["opt_income"].mean() for i in snapshots]
plt.plot(x,y, "bo")

### Calculate income revenue

for df in pricing_pol:
    df["income_diff"] = df["opt_income"] - df["origi_income"]

# 2. Simulation stability

corrs = ["pearson","spearman","kendall","wkendall"]

## a.) Stability of basing node statistics (moved to script)

- stability could be improved by running more independent experiments
- this time (using channel depletions) the experiment is less stable?

## b.) Stability of optimal pricing results

origi_inc = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "origi_income") for snap_id in snapshots])
opt_inc = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "opt_income") for snap_id in snapshots])
income_diff = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "income_diff") for snap_id in snapshots])
opt_delta = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "opt_delta") for snap_id in snapshots])

colors = ['b','g','r','m']
fig, axis = plt.subplots(1,3,figsize=(15,4))
x = snapshots
#incomes
axis[0].set_title("INCOME mean cross correlations")
for i, c in enumerate(corrs):
    axis[0].plot(x,origi_inc[c], colors[i]+'-', label=c+"_orig")
    axis[0].plot(x,opt_inc[c], colors[i]+'--', label=c+"_opt")
axis[0].set_xlabel("snapshot")
axis[0].legend()
#income_diff
axis[1].set_title("INCOME_DIFF mean cross correlations")
for i, c in enumerate(corrs):
    axis[1].plot(x,income_diff[c], colors[i]+'-', label=c)
axis[1].set_xlabel("snapshot")
axis[1].legend()
#opt_delta
axis[2].set_title("OPT_DELTA mean cross correlations")
for i, c in enumerate(corrs):
    axis[2].plot(x,opt_delta[c], colors[i]+'-', label=c)
axis[2].set_xlabel("snapshot")
axis[2].legend()
plt.show()

#### Incomes observations

- taking mean cross correlation of user incomes (original and optimal)
- weighted kendall-tau gives decreased to 0.90 from 0.95 (after including capacity maintenance)
- in case of unweighted spearman and kendall the optimal income correlates better across samples 

#### income_diff observations¶

- taking mean cross correlation of income difference (optimal income - original income)
- weighted kendall-tau decreased to 0.80 from 0.93 (after including capacity maintenance)

#### opt_delta observations¶

- taking mean cross correlation of optimal base fee change
- weighted kendall-tau decreased to 0.70 from 0.90 (after including capacity maintenance)

**CONCLUSION: our base_fee optimization procedure is less efficient (meaningful) after the implementation of capacity maintenance!!!**

# 3. Income revenue analysis

In [ ]:
def cut_into_categories(df):
    #categories = [df[:10], df[10:30], df[30:50], df[50:100]]
    #categories = [df[:20], df[20:50], df[50:100], df[100:200]]
    #categories = [df[:10], df[10:20], df[20:30], df[30:40], df[40:50], df[50:100], df[100:500]]
    categories = [df[:10], df[10:20], df[20:50], df[50:100], df[100:500]]
    #categories = [df[:50], df[50:100], df[100:500]]#, df[200:500]]
    return categories

def merge_categories(categories):
    for i, cat in enumerate(categories):
        cat["category"] = i
    return pd.concat(categories)
    
def show_stats(snap_id, col="income_diff"):
    mean = pricing_pol[snap_id].groupby("node").mean().sort_values("origi_income", ascending=False)
    categories = cut_into_categories(mean)
    return [float(cat[[col]].mean()) for cat in categories]

In [ ]:
x = range(5)
#x = range(7)

# TODO: ez a mérés most teljesen szét van esve!!!

### Visualize each snapshot

df = pd.DataFrame([show_stats(i, "income_diff") for i in snapshots])

plt.Figure(figsize=(10,10))
for i in snapshots:
    plt.plot(x, show_stats(i, "income_diff"), label=i)
plt.yscale("log")
plt.legend()

plt.Figure(figsize=(10,10))
for i in snapshots:
    plt.plot(x, show_stats(i, "opt_traffic"), label=i)
plt.legend()

### Average stats (over all snapshots and samples)

In [ ]:
average_stats = pricing_pol[0].groupby("node").mean().sort_values("origi_income", ascending=False)[["income_diff","origi_income","opt_income","opt_traffic","opt_delta"]]

In [ ]:
pricing_pol[0].head()

In [ ]:
pricing_pol[0].max()

In [ ]:
all_records = pd.concat([pricing_pol[i] for i in snapshots])

In [ ]:
average_stats_all = all_records.groupby("node").mean()

In [ ]:
average_stats = average_stats_all#[average_stats_all["origi_num_trans"]>=10]

In [ ]:
print(len(average_stats_all),len(average_stats))

In [ ]:
average_stats.head()

In [ ]:
average_stats = all_records.groupby("node").mean().sort_values("origi_income", ascending=False)[["income_diff","origi_income","opt_income","opt_traffic","opt_delta"]]

In [ ]:
average_stats_cats = cut_into_categories(average_stats)
mean_diff = [float(cat[["income_diff"]].mean()) for cat in average_stats_cats]
mean_traffic = [float(cat[["opt_traffic"]].mean()) for cat in average_stats_cats]
mean_delta = [float(cat[["opt_delta"]].mean()) for cat in average_stats_cats]

In [ ]:
fig, axis = plt.subplots(1,3,figsize=(15,4))
axis[0].plot(x, mean_diff, 'rx')
axis[0].set_xlabel("income category")
axis[0].set_ylabel("mean income_diff")
axis[1].plot(x, mean_delta, 'g*')
axis[1].set_xlabel("income category")
axis[1].set_ylabel("mean opt_delta")
axis[2].plot(x, mean_traffic, 'bo')
axis[2].set_xlabel("income category")
axis[2].set_ylabel("mean opt_traffic")
plt.show()

#### Mean original income for the selected categories

In [ ]:
[float(cat[["origi_income"]].mean()) for cat in average_stats_cats]

### PROBLEM with outliers...

In [ ]:
merged = merge_categories(average_stats_cats)

In [ ]:
sns.boxplot(data=merged, x="category", y="income_diff", whis=2)

In [ ]:
average_stats.head(10)

### Nodes above 20K SAT (2.5 USD) income_diff (with alpha=2.0 source distribution)

#### 2019-07-04_11:12:08_200000sat_k30000 (ordered by original income)

1. yalls.org - Read and write articles, with Lightning Network micropayments. (195,925.46 USD)
2. ACINQ - ACINQ is one of the leading companies working on Bitcoin scalability. (507,211.38 USD)
3. 1ML.com node ALPHA (89,453.40 USD)
4. LightningPowerUsers.com (313,406.05 USD)
5. LightningTo.Me - Helping to resolve routing and capacity issues (165,531.35 USD)
6. tomjodh.nl - Buy photos with satoshis (8,709.95 USD) - **TODO RECLASSIFY AS merchant!!!**
7. ? - ? (848.05 USD)

#### 2019-07-05_14:39:09_200000sat_k30000 (ordered by original income)

1. ACINQ - ACINQ is one of the leading companies working on Bitcoin scalability. (507,211.38 USD, 783 channel)
2. 1ML.com node ALPHA (89,453.40 USD, 884 channel)
3. LightningPowerUsers.com (313,406.05 USD, 1,266 channel)
4. LightningTo.Me - Helping to resolve routing and capacity issues (165,531.35 USD, 700 channel)
5. tomjodh.nl - Buy photos with satoshis (8,709.95 USD, 47 channel)
6. ? - ? (859.23 USD, 4 channel)
7. nybemotion - ? (318.35 USD, 3 channel)

#### Additional nodes in 2019-07-05_14:39:09_500000sat_k30000

4. Bitrefill.com - Buy Gift Cards with Bitcoin for 30+ supported services (310,369.63 USD, 305 channel) - **TODO RECLASSIFY AS merchant!!!**
6. tady je slushovo: ? (48,872.23 USD, 336 channel)

### Nodes above 20K SAT (2.5 USD) income_diff (with alpha=None source distribution)

#### 2019-08-06_13:42:00_200000sat_k30000_aNone (ordered by original income)

1. yalls.org
2. ACINQ
3. LightningPowerUsers.com
4. tomjodh.nl
5. Node: 0297b368360113e29032 ???
6. nybemotion

#### 2019-08-09_14:31:08_200000sat_k30000_aNone_dropTrue

1. yalls.org
2. ln1.satoshilabs.com (306,117.70 USD - 824 channels) **(NEW)**
3. LightningPowerUsers.com
4. nybemotion (493.61 USD, 6 channel) ???
5. Amazing_Droopy_3 (113.73 USD, 1 channel?) ???

#### 2019-09-01_18:07:51_200000sat_k30000_aNone_dropTrue (WITH capacity maintenance)

1. LightningPowerUsers.com (237,653 USD, 1261 channels)
2. 1ML.com node ALPHA (64,630 USD, 889 channels)
3. CoinGate (147,272 USD, 462 channels) - CoinGate is the first large cryptocurrency payment gateway to introduce Lightning Network payments to hundreds of merchants with no technical knowledge required
4. lndhub.ru (19,066 USD, 248 channels) - increase connectivity
5. BeerselLightningNode (13,246 USD, 84 channels)
6. BitKassa LN34961 (13,631 USD, 47) - some kind of payment API and BTC tech company
7. Serving Beer (2,107 USD, 13 channels) - is this really a merchant???

average_stats[average_stats["income_diff"]>20000]